# Full Resolution - Several (3) cones

From user data `BodyName`, `Date`, `Observed Altitude`.


In [37]:
%classpath add jar "../build/libs/MPS-1.0-all.jar"

Some utilities - class and methods, used below:

In [38]:
import calc.calculation.AstroComputerV2;

import java.text.SimpleDateFormat;
import java.util.Calendar;
import java.util.TimeZone;

import calc.CelestialDeadReckoning;
import calc.GeoPoint;
import calc.GeomUtil;
import mps.MPSToolBox;

import calc.calculation.nauticalalmanacV2.Star;

private final static SimpleDateFormat SDF_UTC = new SimpleDateFormat("yyyy-MMM-dd HH:mm:ss 'UTC'");
// static {
    SDF_UTC.setTimeZone(TimeZone.getTimeZone("Etc/UTC"));
//}
private final static SimpleDateFormat DURATION_FMT = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss");
// static {
    DURATION_FMT.setTimeZone(TimeZone.getTimeZone("Etc/UTC"));
// }

// Point of reference, user's position, used on option.
double userLatitude = 47.677667;
double userLongitude = -3.135667;

private static class BodyData {
    String bodyName;
    Date date;
    Double gha;
    Double decl;
    Double obsAlt;

    public BodyData() {
    }
    public BodyData(String bodyName, Date date, Double gha, Double decl, Double obsAlt) {
        this.bodyName = bodyName;
        this.date = date;
        this.gha = gha;
        this.decl = decl;
        this.obsAlt = obsAlt;
    }

    public String getBodyName() {
        return bodyName;
    }

    public void setBodyName(String bodyName) {
        this.bodyName = bodyName;
    }

    public Date getDate() {
        return date;
    }

    public void setDate(Date date) {
        this.date = date;
    }

    public Double getGha() {
        return gha;
    }

    public void setGha(Double gha) {
        this.gha = gha;
    }

    public Double getDecl() {
        return decl;
    }

    public void setDecl(Double decl) {
        this.decl = decl;
    }

    public Double getObsAlt() {
        return obsAlt;
    }

    public void setObsAlt(Double obsAlt) {
        this.obsAlt = obsAlt;
    }
}

/**
 *
 * @param duration, format YYYY-MM-DDTHH:MI:SS
 *                         |    |  |  |  |  |
 *                         |    |  |  |  |  17
 *                         |    |  |  |  14
 *                         |    |  |  11
 *                         |    |  8
 *                         |    5
 *                         0
 * @return The corresponding UTC Date
 * @throws Exception
 */
protected Calendar parseDuration(String duration) throws Exception {
    int year = Integer.parseInt(duration.substring(0, 4));
    int month = Integer.parseInt(duration.substring(5, 7));
    int day = Integer.parseInt(duration.substring(8, 10));
    int hours = Integer.parseInt(duration.substring(11, 13));
    int minutes = Integer.parseInt(duration.substring(14, 16));
    int seconds = Integer.parseInt(duration.substring(17));

    Calendar date = Calendar.getInstance(TimeZone.getTimeZone("Etc/UTC")); // Now
    date.set(Calendar.YEAR, year);
    date.set(Calendar.MONTH, month - 1);
    date.set(Calendar.DAY_OF_MONTH, day);
    date.set(Calendar.HOUR_OF_DAY, hours); // and not just HOUR !!!!
    date.set(Calendar.MINUTE, minutes);
    date.set(Calendar.SECOND, seconds);

    return date;
}

protected boolean validateBody(String body) {
    boolean ok = true;

    switch (body) {
        case "Sun":
        case "Moon":
        case "Venus":
        case "Mars":
        case "Jupiter":
        case "Saturn":
            ok = true;
            break;
        default:
            Star star = Star.getStar(body);
            ok = star != null;
            break;
    }
    return ok;
}

protected BodyData computeBodyData(String body, String dateDuration, double obsAlt) throws Exception {

    BodyData bodyData = new BodyData();
    bodyData.setBodyName(body);
    bodyData.setObsAlt(obsAlt);

    AstroComputerV2 ac = new AstroComputerV2();

    Calendar date = parseDuration(dateDuration);
    bodyData.setDate(date.getTime());
    System.out.printf("Calculation launched for %s at %s\n", body, SDF_UTC.format(date.getTime()));

    if (!validateBody(body)) {
        throw new Exception(String.format("Invalid body [%s]", body));
    // } else {
    //    System.out.printf("%s OK\n", body);
    }

    ac.calculate(date.get(Calendar.YEAR),
            date.get(Calendar.MONTH) + 1,
            date.get(Calendar.DAY_OF_MONTH),
            date.get(Calendar.HOUR_OF_DAY), // and not just HOUR !!!!
            date.get(Calendar.MINUTE),
            date.get(Calendar.SECOND),
            true);

    double deltaT = ac.getDeltaT(); // Unused for now

    double gha, decl;

    switch (body) {
        case "Sun":
            gha = ac.getSunGHA();
            decl = ac.getSunDecl();
            break;
        case "Moon":
            gha = ac.getMoonGHA();
            decl = ac.getMoonDecl();
            break;
        case "Venus":
            gha = ac.getVenusGHA();
            decl = ac.getVenusDecl();
            break;
        case "Mars":
            gha = ac.getMarsGHA();
            decl = ac.getMarsDecl();
            break;
        case "Jupiter":
            gha = ac.getJupiterGHA();
            decl = ac.getJupiterDecl();
            break;
        case "Saturn":
            gha = ac.getSaturnGHA();
            decl = ac.getSaturnDecl();
            break;
        default:
            ac.starPos(body);
            // final double sha = ac.getStarSHA(body);
            gha = ac.getStarGHA(body);
            decl = ac.getStarDec(body);
            break;
    }
    if (false) {
        CelestialDeadReckoning dr = MPSToolBox.calculateDR(gha, decl, userLatitude, userLongitude).calculate(); // All angles in degrees
        double he = dr.getHe();
        System.out.printf("For %s at %s, ObsAlt should be %f (%s)\n", body, SDF_UTC.format(date.getTime()), he, GeomUtil.decToSex(he, GeomUtil.SHELL, GeomUtil.NONE));
    }
    bodyData.setGha(gha);
    bodyData.setDecl(decl);

    return bodyData;
}

Populate the body data list with the following
```
Body=Mars;Date=2025-10-07T15:36:00; ObsAlt=21º56.54'
Body=Venus;Date=2025-10-07T15:36:00;ObsAlt=14.014
#                                          14.014 = 14º0.84'
Body=Altair;Date=2025-10-07T15:36:00;ObsAlt=32º28.63'
```
For PlayGround09, this comes from a text file passed at runtime.

In [39]:
List<BodyData> listBodyData = new ArrayList<>();

listBodyData.add(computeBodyData("Venus",  "2025-10-07T15:36:00", GeomUtil.sexToDec("14º0.84'")));
listBodyData.add(computeBodyData("Mars",   "2025-10-07T15:36:00", GeomUtil.sexToDec("21º56.54'")));
listBodyData.add(computeBodyData("Altair", "2025-10-07T15:36:00", GeomUtil.sexToDec("32º28.63'")));

Calculation launched for Venus at 2025-Oct-07 15:36:00 UTC
Calculation launched for Mars at 2025-Oct-07 15:36:00 UTC
Calculation launched for Altair at 2025-Oct-07 15:36:00 UTC


true

In [40]:
import java.text.NumberFormat;

// Good to go ?
System.out.printf("We have %d bodies.\n", listBodyData.size());
listBodyData.forEach(bd-> System.out.printf("%s:\t ObsAlt: %s (%f),\t GHA: %s (%f),\t Decl: %s (%f)\n",
                                            bd.getBodyName(),
                                            GeomUtil.decToSex(bd.getObsAlt(), GeomUtil.SHELL, GeomUtil.NONE),
                                            bd.getObsAlt(),
                                            GeomUtil.decToSex(bd.getGha(), GeomUtil.SHELL, GeomUtil.NONE),
                                            bd.getGha(),
                                            GeomUtil.decToSex(bd.getDecl(), GeomUtil.SHELL, GeomUtil.NS),
                                            bd.getDecl()));
 
List<MPSToolBox.ConesIntersection> conesIntersectionList = new ArrayList<>();

final long before = System.currentTimeMillis();

/// Cones and Co here. All permutations.
int nbProcess = 0;
for (int i=0; i<listBodyData.size(); i++) {
    for (int j=0; j<listBodyData.size(); j++) {
        if (i != j) {
            System.out.printf("[%d, %d], %s and %s\n", i, j, listBodyData.get(i).getBodyName(), listBodyData.get(j).getBodyName());
            String bodyOne = listBodyData.get(i).getBodyName();
            double altOne = listBodyData.get(i).getObsAlt(); // saturnObsAlt;
            double ghaOne = listBodyData.get(i).getGha();    // saturnGHA;
            double declOne = listBodyData.get(i).getDecl();  // saturnDecl;
            Date dateOne = listBodyData.get(i).getDate();    // date.getTime();

            String bodyTwo = listBodyData.get(j).getBodyName();
            double altTwo = listBodyData.get(j).getObsAlt(); // jupiterObsAlt;
            double ghaTwo = listBodyData.get(j).getGha();    // jupiterGHA;
            double declTwo = listBodyData.get(j).getDecl();  // jupiterDecl;
            Date dateTwo = listBodyData.get(j).getDate();    // date.getTime();

            int nbIter = 4;
            boolean reverse = false;
            boolean verbose = true;

            if (verbose) {
                System.out.println("------------------------------------------------");
                System.out.printf("Starting resolve process with:\n" +
                                "Time1: %s, Alt1: %s, GHA1: %s, Decl1: %s\n" +
                                "Time2: %s, Alt2: %s, GHA2: %s, Decl2: %s\n",
                        DURATION_FMT.format(dateOne),
                        GeomUtil.decToSex(altOne, GeomUtil.SHELL, GeomUtil.NONE),
                        GeomUtil.decToSex(ghaOne, GeomUtil.SHELL, GeomUtil.NONE),
                        GeomUtil.decToSex(declOne, GeomUtil.SHELL, GeomUtil.NS),
                        DURATION_FMT.format(dateTwo),
                        GeomUtil.decToSex(altTwo, GeomUtil.SHELL, GeomUtil.NONE),
                        GeomUtil.decToSex(ghaTwo, GeomUtil.SHELL, GeomUtil.NONE),
                        GeomUtil.decToSex(declTwo, GeomUtil.SHELL, GeomUtil.NS));
                System.out.println("------------------------------------------------");
            }
            // Ephemeris and Altitudes OK, let's proceed.
            double firstZStep = 0.1d;  // More than 0.1 not good enough...

            // Now, find the intersection(s) of the two cones...
            List<GeoPoint> closests = MPSToolBox.resolve2Cones(dateOne, altOne, ghaOne, declOne,
                                                               dateTwo, altTwo, ghaTwo, declTwo,
                                                               firstZStep, nbIter, reverse, verbose);

            if (closests != null) {
                final double d1 = GeomUtil.haversineNm(closests.get(0), closests.get(1));
                final double d2 = GeomUtil.haversineNm(closests.get(2), closests.get(3));
                System.out.printf("%d - %s & %s\n", ++nbProcess, bodyOne, bodyTwo);
                System.out.printf("After %d iterations:\n", nbIter);
                System.out.printf("1st position between %s (%s) and %s (%s), dist %.02f nm.\n", closests.get(0), closests.get(0).toNumericalString(), closests.get(1), closests.get(1).toNumericalString(), d1);
                System.out.printf("2nd position between %s (%s) and %s (%s), dist %.02f nm.\n", closests.get(2), closests.get(2).toNumericalString(), closests.get(2), closests.get(2).toNumericalString(), d2);
                // For later
                conesIntersectionList.add(new MPSToolBox.ConesIntersection(bodyOne, bodyTwo,
                                                                           closests.get(0), closests.get(1),
                                                                           closests.get(2), closests.get(3)));
            } else {
                System.out.println("Oops ! Not found...");
            }

        }
    }
}
System.out.printf("End of permutations, %d intersections\n", conesIntersectionList.size());
final long after = System.currentTimeMillis();
System.out.println("-----------------------------");
System.out.printf("Full Intersection Calculation took %s ms (System Time)\n", NumberFormat.getInstance().format(after - before));

We have 3 bodies.
Venus:	 ObsAlt:  14º00.84' (14.014000),	 GHA:  77º04.14' (77.068942),	 Decl: 4º22.30'N (4.371674)
Mars:	 ObsAlt:  21º56.54' (21.942333),	 GHA:  32º35.17' (32.586194),	 Decl: 15º05.10'S (-15.084996)
Altair:	 ObsAlt:  32º28.63' (32.477167),	 GHA:  312º31.51' (312.525102),	 Decl: 8º56.33'N (8.938912)
[0, 1], Venus and Mars
------------------------------------------------
Starting resolve process with:
Time1: 2025-10-07T15:36:00, Alt1:  14º00.84', GHA1:  77º04.14', Decl1: 4º22.30'N
Time2: 2025-10-07T15:36:00, Alt2:  21º56.54', GHA2:  32º35.17', Decl2: 15º05.10'S
------------------------------------------------
1st Intersection: Found dist = 1.153, zStep=0.010, between 47º42.06'N / 3º08.48'W (Z=41.80) and 47º40.93'N / 3º08.83'W (Z=20.90)
2nd Intersection: Found dist = 2.224, zStep=0.010, between 70º05.43'S / 99º30.65'W (Z=187.70) and 70º07.41'S / 99º27.69'W (Z=199.70)
-- (1st : between 47º42.06'N / 3º08.48'W (Z=41.80) and 47º40.93'N / 3º08.83'W (Z=20.90))
Loop 1 - Smallest

java.io.PrintStream@331295bf

In [41]:
// Now process all intersections...
try {
    GeoPoint avgPoint = MPSToolBox.processIntersectionsList(conesIntersectionList, false);
    System.out.printf("Found (avg) intersection at %s\n", avgPoint);
    if (false) {
        GeoPoint original = new GeoPoint(userLatitude, userLongitude);
        System.out.printf("=> Compare to original position: %s\n", original);
        System.out.printf("==> Difference/offset: %.02f nm\n", GeomUtil.haversineNm(original, avgPoint));
    }
} catch (MPSToolBox.NotEnoughIntersectionsException mei) {
    mei.printStackTrace();
}

Found (avg) intersection at 47º40.65'N / 3º08.14'W
